In [3]:
%matplotlib inline

import matplotlib
import autograd.numpy as np
import matplotlib.pyplot as plt
import random
import math
from autograd import grad

def generateChevronData():
    xBounds = [-50, 50]
    yBounds = [-50, 50]
    totalPoints = 100
    
    points = []
    targets = []
    
    for i in range(0, totalPoints):
        x = random.randint(xBounds[0], xBounds[1])
        y = random.randint(yBounds[0], yBounds[1])
        
        if x >= y and x <= -y:
            points.append([x/50.0,y/50.0])
            targets.append(0)
        else:
            points.append([x/50.0,y/50.0])
            targets.append(1)
        
    return np.array(points), np.array(targets)
    
def plotScatter(points):
    xs = [x[0] for x in points]
    ys = [y[1] for y in points]
    
    plt.scatter(xs, ys)

In [4]:
def sigmoid(phi):
    return 1.0/(1.0 + np.exp(-phi))

def MSE(weights):
    predictions = logisticPrediction(weights, points)
    w = np.full((len(predictions)), np.log(1/2)) # CONSTANT
    r = responsibility(weights, points)
#     print(r)
    return -np.sum( r * ((targets*np.log(predictions) + (1-targets)*np.log(1-predictions))) + (1-r) * w)

def logisticPrediction(weights, p):
    return np.array(list(map(lambda x: predict(weights, x), p))) 
    
def predict(weights, inputs):
    n = np.array([weights[0], weights[1]])
    i = np.array([weights[2] - inputs[0], weights[3] - inputs[1]])
    return sigmoid(np.dot(n, i))

def responsibility(weights, points):
    r = weights[4]
    a = np.array([weights[2], weights[3]])
    
    dif = np.array(list(map(lambda x: x - a, points)))
    s = np.array(list(map(lambda x: np.sum(np.power(x, 2)), dif)))
    d = np.sqrt(s)
    
    return 1/(1 + np.power(np.e, (10*(d - r))))

In [5]:
def computeGradient(weights, example, target):
    prediction = predict(weights, example)
    r = responsibility(weights, [example])[0]
    dE_dO = computeErrorDifferential(r, prediction, target)
    dE_dr = computeResponsibilityDifferential(weights, example)
    
    dO_dZ = prediction * (1-prediction)
    
#     dZ_d0 = example[0]
    dZ_d1 = (weights[2] - example[0])
    dZ_d2 = (weights[3] - example[1])
    dZ_d3 = weights[0]
    dZ_d4 = weights[1]
    
    dE_dZ = dE_dO * dO_dZ
    
    grad = np.zeros(len(weights))#[0.0, 0.0, 0.0]
#     grad[0] = dZ_d0 * dE_dZ
    grad[0] = dZ_d1 * dE_dZ
    grad[1] = dZ_d2 * dE_dZ
    grad[2] = dZ_d3 * dE_dZ
    grad[3] = dZ_d4 * dE_dZ
    grad[4] = -dE_dr * ((target*np.log(prediction) + (1-target)*np.log(1-prediction)) - np.log(1/2))
#     print(((target*np.log(prediction) + (1-target)*np.log(1-prediction))))
    
    return grad

def computeErrorDifferential(r, prediction, target):
    return -r * (target - prediction)/(prediction - prediction**2)

def computeResponsibilityDifferential(weights, example):
    r = weights[4]
    a = np.array([weights[2], weights[3]])
    
    dif = example - a
    s = np.sum(np.power(dif, 2))
    d = np.sqrt(s)
    
    return (10 * np.power(np.e, (d + r)))/np.power(np.power(np.e, 10*d) + np.power(np.e, 10*r), 2)

In [22]:
def trainBoundaryHunter():
    weights = np.array([0.0, 0.0, 0.0, 0.0, 0.3])
    
    print("Initial Loss: ", MSE(weights))
    for i in range(0, 10000):
        weights = computeStep(weights)
    
        if i % 1000 == 0:
            print("Loss [i = " + str(i) + "]: " + str(MSE(weights)))
            print(weights)
            
    print("Trained Loss: ", MSE(weights))    
    print("Weights: ", weights)
    return weights

def computeStep(weights):
#     totalG = np.zeros(len(weights))
#     totalE = 0
#     for i in range(0, len(points)):
#         g = computeGradient(weights, points[i], targets[i])
#         totalG += g     
        
#     totalG = totalG * (1/len(points))
    
    grad = checkGrad(0.00001, 0.00001, weights)
    weights -= grad * 0.01
    return weights

def checkGrad(pterb, threshold, weights):
    grad = np.zeros(len(weights))
    for i in range(0, len(weights)):
        p = np.zeros(len(weights))
        p[i] = pterb
        
        SSEBefore = MSE(weights)
        SSEAfter = MSE(weights + p)
        
        grad[i] = (SSEAfter - SSEBefore)/pterb
        

    return grad

#     dif = np.absolute(computedGrad - grad)
#     for d in dif:
#         if d > threshold:
#             print("ERROR")
    

In [ ]:
random.seed(1234)
points, targets = generateChevronData()

plt.axis([-1.5, 1.5, -1.5, 1.5])

# Plot points on graph
c1 = []
c2 = []

for i in range(0, len(points)):
    if targets[i] == 0:
        c1.append(points[i])
    else:
        c2.append(points[i])

print("Type 0: ", len(c1))
print("Type 1: ", len(c2))
        
plotScatter(c1)
plotScatter(c2)

weights = trainBoundaryHunter()

# plt.scatter(weights[1], weights[2])
plt.scatter(weights[2], weights[3])

n = np.array([weights[0] * weights[2] + weights[1] * weights[3], 
              -weights[0], 
              -weights[1]])

byas = -1 * n[0]/n[2]
Xcoef = -1 * n[1]/n[2]

x = np.linspace(-1.5, 1.5, 500)
y = np.linspace(-1.5, 1.5, 500)
X, Y = np.meshgrid(x,y)
F = ((X - weights[2]))**2 + ((Y - weights[3]))**2 - weights[4]**2
plt.contour(X,Y,F,[0])

# print()
# print(n)
# print("\nLine")
# print("B: " + str(byas))
# print("XCoef: " + str(Xcoef))

plt.plot([-1.0, 1.0], [-1*Xcoef + byas, Xcoef + byas], 'k-')
plt.gca().set_aspect('equal')

plt.show()

Type 0:  35
Type 1:  65
Initial Loss:  69.314718056
Loss [i = 0]: 69.3103676917
[-0.00215269 -0.00623677  0.          0.          0.3       ]
Loss [i = 1000]: 35.3933455211
[-0.77048669 -4.1126549   0.00816401 -0.72869434  2.6680832 ]
Loss [i = 2000]: 35.3931594421
[-0.77043506 -4.11271294  0.00892972 -0.72883699  2.69797773]
Loss [i = 3000]: 35.3930354639
[-0.7704157  -4.1127454   0.00924151 -0.72889466  2.72100335]
Loss [i = 4000]: 35.3929529432
[-0.77040307 -4.1127668   0.00944547 -0.72893237  2.73972681]
Loss [i = 5000]: 35.392894086
[-0.77039418 -4.11278196  0.00958928 -0.72895894  2.75550291]
Loss [i = 6000]: 35.3928499966
[-0.77038759 -4.11279326  0.00969611 -0.72897868  2.76913328]
Loss [i = 7000]: 35.3928157477
[-0.77038251 -4.11280201  0.00977859 -0.72899392  2.78113159]
Loss [i = 8000]: 35.3927883744
[-0.77037847 -4.11280899  0.00984419 -0.72900604  2.79184676]
Loss [i = 9000]: 35.3927659946
[-0.77037518 -4.11281468  0.0098976  -0.7290159   2.80152663]
